### 1. Generating Data

$y_{(\text{n}\times 1)} = $
$\text{X}_{(\text{n}\times\text{p})}$
$\beta^{\text{true}}_{(\text{p}\times\text{1})}$
$+\epsilon_{(\text{n}\times\text{1})}$

In [147]:
import navigator_updater.static.css
import numpy as np
from numpy import linalg
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

def get_W(p):
    list = []
    list.append(np.zeros((p,p)))
    list.append(np.eye(p))
    return list

def transformation(y, X, W, ev):    # ev: eigenvalue
    # dimension of z: n+p
    z = np.vstack((y, np.zeros((len(W),1) )))

    # dimension of U: (n+p) x p
    U = np.vstack((X, -np.sqrt(ev)*W))
    return z, U

def Initialization(n, p):
    beta_true = np.ones((p, 1))
    X = np.random.normal(0, 1, size=(n,p))
    noise = np.random.normal(0, 1, size=(n, 1))
    # print(f'beta_ture shape: {beta_true.shape}')
    # print(X.shape)
    # print(noise.shape)
    y = np.matmul(X, beta_true)+noise
    # print(y)
    # print(y.shape)
    return y, X

### 2. Estimation

In [148]:
def Cholesky(y, X):
    L = np.linalg.cholesky(np.matmul(np.transpose(X),X))
    # print(L)

    theta = np.matmul(np.transpose(X), y)
    theta = np.matmul(np.linalg.inv(L), theta)

    beta_estimate = np.linalg.inv(np.transpose(L))
    beta_estimate = np.matmul(beta_estimate, theta)
    # print(f'the estimate of beta estimated by Cholesky-BFS is: \n{beta_estimate}')
    return beta_estimate

def QR_decomposition(y,X):
    Q, R = np.linalg.qr(X)
    beta_estimate = np.matmul(np.transpose(Q), y)
    beta_estimate = np.matmul(np.linalg.inv(R),beta_estimate)
    # print(f'the estimate of beta estimated by QR-decomposition is: \n{beta_estimate}')
    return beta_estimate

def SVD(y, X):

    U, S, VH = linalg.svd(X)

    Delta = np.zeros((len(U),len(VH)))
    Delta[:len(S), :len(S)] = np.diag(S)

    D = np.matmul(np.transpose(Delta),Delta)
    D = np.linalg.inv(D)

    ev_decomposition = np.matmul(D, VH)
    ev_decomposition = np.matmul(np.transpose(VH),ev_decomposition)
    beta_estimate = np.matmul(np.transpose(X),y)
    beta_estimate = np.matmul(ev_decomposition, beta_estimate)

    # print(f'the estimate of beta estimated by SVD is: \n{beta_estimate}')
    return beta_estimate

#### 3. Runtime of the three algorithms
* 其中只有p, W 改變，兩個迴圈


In [149]:
import time
n = 1100; ev = 0.1
p_list = [10, 1000]

import pandas as pd

methods = {'algorithms': ["Cholesky", "QR-decomposition", "SVD"]}
table = pd.DataFrame(data=methods)

# for p in p_list:
for i in range(1, 4):
    p = 10**i
    y, X = Initialization(n, p)
    W_list = get_W(p)
    # print(W_list)
    for W in W_list:
        z, U = transformation(y, X, W, ev)
        result = []
        start_time = time.time()
        Cholesky(z, U)
        Cholesky_time = time.time()-start_time
        result.append(Cholesky_time)
        # print(f"Solve Ridge with Cholesky (second): {Cholesky_time}")

        start_time = time.time()
        QR_decomposition(z, U)
        QR_time = time.time()-start_time
        result.append(QR_time)
        # print(f"Solve Ridge with QR (second): {QR_time}")

        start_time = time.time()
        SVD(z, U)
        SVD_time = time.time()-start_time
        result.append(SVD_time)
        # print(f"Solve Ridge with SVD (second): {SVD_time}")
        if np.max(W) == 1:
            matirx_type = "I"
        else:
            matirx_type = "O"
        table["n:{n}, p:{p}, W:{matrix_type}{shape}, lambda = {ev}".format(n=n, p=p, matrix_type=matirx_type, shape=W.shape, ev=ev)] = result

In [155]:
table

,algorithms,"n:1100, p:10, W:O(10, 10), lambda = 0.1","n:1100, p:10, W:I(10, 10), lambda = 0.1","n:1100, p:100, W:O(100, 100), lambda = 0.1","n:1100, p:100, W:I(100, 100), lambda = 0.1","n:1100, p:1000, W:O(1000, 1000), lambda = 0.1","n:1100, p:1000, W:I(1000, 1000), lambda = 0.1"
0,Cholesky,0.000993,0.000492,0.036699,0.007435,0.232122,0.102672
1,QR-decomposition,0.001263,0.000495,0.018353,0.017361,0.218736,0.195140
2,SVD,0.008930,0.010916,0.051088,0.041118,0.815256,0.824782


In [156]:
numpy_table = table.drop(columns=["algorithms"]).to_numpy()

numpy_table

array([[0.001, 0.000, 0.037, 0.007, 0.232, 0.103],
       [0.001, 0.000, 0.018, 0.017, 0.219, 0.195],
       [0.009, 0.011, 0.051, 0.041, 0.815, 0.825]])